In [5]:
from os.path import exists
import pandas as pd

In [21]:
train_df=pd.read_csv('train.csv')
train_df['store'].dtype

# take a sample of 19,000 data items
df=train_df.sample(n=19000,random_state=0)
# convert store and item columns into string
df["item"]=df['item'].astype(str)
df["store"]=df['store'].astype(str)
df["date"]=pd.to_datetime(df['date'])

# sort in terms of dates

df.sort_values('date',inplace=True)
df.reset_index(drop=True,inplace=True)
df.head()

In [27]:
# split time series data train dataset 2013-2016   and 2017 data into test dataset
train=df[df['date']<"2017-01-01"]
test=df[df['date']>="2017-01-01"]
test

,date,store,item,sales
15213,2017-01-01,1,44,32
15214,2017-01-01,4,25,64
15215,2017-01-01,3,15,89
15216,2017-01-01,9,50,67
15217,2017-01-01,4,37,29
...,...,...,...,...
18995,2017-12-31,8,10,87
18996,2017-12-31,1,11,44
18997,2017-12-31,10,49,42
18998,2017-12-31,3,21,46


In [28]:
# split trainand test into features and labels
train_target=train['sales']
train_features=train.drop(columns='sales')

test_target=test['sales']
test_features=test.drop(columns='sales')

In [48]:
# we will enrich our features using upgini library
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType

enricher=FeaturesEnricher(
    search_keys={
        'date':SearchKey.DATE,
    },
    cv=CVType.time_series
)
enricher.fit(train_features,train_target,eval_set=[(test_features,test_target)])

In [ ]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric

model=CatBoostRegressor(verbose=False,allow_writing_files=False,random_state=0)

enricher.calculate_metrics(
train_features,train_target,
eval_set=[(test_features,test_target)],
estimator=model,
scoring="mean_absolute_percentage_error")

In [ ]:
enriched_train_features=enricher.transform(train_features,keep_input=True)
enriched_test_features=enricher.transform(test_features,keep_input=True)
enriched_train_features.head()

[                                                            ] 0% Checking dataset...

('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
We apologize for the inconvenience as we were experiencing technical issues with cloud infrastructure today.
We are actively working on a fix and expect to have the issue resolved in the next couple of hours.

('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
We apologize for the inconvenience as we were experiencing technical issues with cloud infrastructure today.
We are actively working on a fix and expect to have the issue resolved in the next couple of hours.

